In [77]:
import pandas as pd
import tqdm
import keras

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [114]:
# Воссоздадим точно такую же модель, включая веса и оптимизатор:
model = keras.models.load_model('best_model.h5')
model.summary()

c:\users\shchukin\appdata\local\programs\python\python36\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [110]:
sales = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Продажи')
sales = sales.set_index('Material')

ap = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Магазины')
ap = ap.set_index('Material')

stock = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Остатки')
stock = stock.set_index('Material')

price = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Цены')
price = price.set_index('Material')


corr_df = pd.read_excel('material_clusters.xlsx')

In [4]:
input_data = pd.DataFrame()

for x in tqdm.tqdm(sales.index[:]):
    material = x

    df = pd.concat([sales.loc[material].shift(-1).rename('sales_1'),\
               sales.loc[material].shift(-2).rename('sales_2'),\
               sales.loc[material].shift(-12).rename('sales_12'),\
               sales.loc[material].subtract(sales.loc[material].shift(-1)).rename('sales_diff_1'),\
               sales.loc[material].subtract(sales.loc[material].shift(-2)).rename('sales_diff_2'),\
               sales.loc[material].subtract(sales.loc[material].shift(-12)).rename('sales_diff_12'),\
               stock.loc[material].shift(-1).rename('stock_1'),\
               stock.loc[material].shift(-2).rename('stock_2'),\
               stock.loc[material].subtract(stock.loc[material].shift(-1)).rename('stock_diff_1'),\
               stock.loc[material].subtract(stock.loc[material].shift(-2)).rename('stock_diff_2'),\
               price.loc[material].shift(-1).rename('price_1'),\
               price.loc[material].shift(-1).rename('price_2'),\
               ap.loc[material].shift(-1).rename('ap_1'),\
               ap.loc[material].shift(-1).rename('ap_2')],\
              axis=1)

    df = df.dropna()

    df.loc[:,'material'] = material
    
    input_data = input_data.append(df, sort=False)
    
input_data.loc[:,'month'] = full_train_data.index.month

  1%|▎                                       | 54/7032 [00:04<09:11, 12.65it/s]


KeyboardInterrupt: 

In [100]:
data_df = pd.melt(sales.reset_index(), id_vars=['Material'], value_vars=list(sales.columns), var_name='date', value_name='sales_1').sort_values(by=['Material','date'], ascending=False).shift(-1)
data_df = pd.merge(data_df, pd.melt(stock.reset_index(), id_vars=['Material'], value_vars=list(stock.columns), var_name='date', value_name='stock_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df = pd.merge(data_df, pd.melt(price.reset_index(), id_vars=['Material'], value_vars=list(price.columns), var_name='date', value_name='price_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df = pd.merge(data_df, pd.melt(ap.reset_index(), id_vars=['Material'], value_vars=list(ap.columns), var_name='date', value_name='ap_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df.loc[:,'sales_2'] = data_df['sales_1'].shift(-1)
data_df.loc[:,'sales_12'] = data_df['sales_1'].shift(-11)
data_df.loc[:,'sales_diff_1'] = data_df['sales_1'] - data_df['sales_1'].shift(-1)
data_df.loc[:,'sales_diff_2'] = data_df['sales_1'] - data_df['sales_1'].shift(-2)
data_df.loc[:,'sales_diff_12'] = data_df['sales_1'] - data_df['sales_1'].shift(-11)
data_df.loc[:,'stock_2'] = data_df['stock_1'].shift(-1)
data_df.loc[:,'stock_diff_1'] = data_df['stock_1'] - data_df['sales_1'].shift(-1)
data_df.loc[:,'stock_diff_2'] = data_df['stock_1'] - data_df['sales_1'].shift(-2)
data_df.loc[:,'price_2'] = data_df['price_1'].shift(-1)
data_df.loc[:,'ap_2'] = data_df['ap_1'].shift(-1)

data_df.loc[:,'date'] = data_df['date'] + pd.offsets.MonthOffset(1)

data_df = data_df.set_index('date')
    
data_df.loc[:,'month'] = data_df.index.month

In [111]:
data_df['2017-12-01'].head()

,Material,sales_1,stock_1,price_1,ap_1,sales_2,sales_12,sales_diff_1,sales_diff_2,sales_diff_12,stock_2,stock_diff_1,stock_diff_2,price_2,ap_2,month
date,,,,,,,,,,,,,,,,
2017-12-01,327636.0,5.0,62.8125,378653.542477,123.0,0.0,0.0,5.0,5.0,5.0,0.0,62.8125,62.8125,0.0,0.0,12.0
2017-12-01,327635.0,1.0,43.0666,396220.917139,81.0,0.0,0.0,1.0,1.0,1.0,0.0,43.0666,43.0666,0.0,0.0,12.0
2017-12-01,308237.0,6.0,988.3000,483734.631543,272.0,0.0,0.0,6.0,6.0,6.0,0.0,988.3000,988.3000,0.0,0.0,12.0
2017-12-01,308220.0,2.0,8.8888,240888.618493,22.0,0.0,0.0,2.0,2.0,2.0,0.0,8.8888,8.8888,0.0,0.0,12.0
2017-12-01,308186.0,1.0,9.4444,180459.659734,23.0,0.0,0.0,1.0,1.0,1.0,0.0,9.4444,9.4444,0.0,0.0,12.0


In [146]:
forecast_df = data_df['2017-11-01']

# Цепляем кластер к товару
forecast_df = forecast_df.merge(corr_df.loc[:,['Material','clusters']], how='left', left_on='Material', right_on='Material')

X = forecast_df[[col for col in train_data.columns if col not in ['sales', 'Material']]][50:]

Y = forecast_df.sales_1.shift(1)[50:].to_frame()

scaler = MinMaxScaler()

print(scaler.fit(X))
print(scaler.fit(Y))

xscale=scaler.transform(X)
yscale=scaler.transform(Y)

c:\users\shchukin\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))
MinMaxScaler(copy=True, feature_range=(0, 1))


In [147]:
prediction = model.predict(xscale)

accuracy = 1 - sum(abs(scaler.inverse_transform(prediction) - scaler.inverse_transform(yscale))) / sum(scaler.inverse_transform(yscale))

print('Total accuracy = %.2f%%' % (accuracy * 100))

Total accuracy = -1447530.70%


In [150]:
prediction

array([[ 120.96049 ],
       [  50.640415],
       [  90.9202  ],
       ...,
       [2468.8123  ],
       [ 304.66037 ],
       [ 750.08624 ]], dtype=float32)

In [149]:
scaler.inverse_transform(yscale)

array([[20.],
       [16.],
       [34.],
       ...,
       [ 5.],
       [ 8.],
       [ 4.]])